In [25]:
import pandas as pd
import datetime
import csv
import numpy as np
import os
import scipy as sp
import xgboost as xgb
import itertools
import operator
import warnings
warnings.filterwarnings("ignore")
path = '~/.kaggle/competitions/rossmann-store-sales'

In [26]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.base import TransformerMixin
from sklearn import cross_validation
from matplotlib import pylab as plt

plot = True

goal = 'Sales'
myid = 'Id'

### 定义损失函数

In [28]:
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

def rmspe_xg(yhat, y):
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe

In [8]:
store = pd.read_csv(path + '/store.csv')

In [9]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [10]:
train_dt = pd.read_csv(path + "/train.csv")

In [11]:
train_dt.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [12]:
test_df = pd.read_csv(path + "/test.csv")

In [13]:
test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


### 加载数据

In [40]:
def load_data():
    store = pd.read_csv(path + '/store.csv')
    train_org = pd.read_csv(path + "/train.csv", dtype={'StateHoliday':pd.np.string_})
    test_org = pd.read_csv(path + "/test.csv", dtype={'StateHoliday':pd.np.string_})
    train = pd.merge(train_org, store, on='Store', how='left')
    test = pd.merge(test_org, store, on='Store', how='left')
    features = test.columns.tolist()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    features_numeric = test.select_dtypes(include=numerics).columns.tolist()
    features_non_numeric = [f for f in features if f not in features_numeric]
    return (train,test,features,features_non_numeric)

### 特征处理/工程

In [55]:
def process_data(train,test,features,features_non_numeric):
    """
    Feature engineering and selection.
    """
    # # FEATURE ENGINEERING
    train = train[train['Sales'] > 0]

    for data in [train,test]:
        # year month day
        data['year'] = data.Date.apply(lambda x: x.split('-')[0])
        data['year'] = data['year'].astype(float)
        data['month'] = data.Date.apply(lambda x: x.split('-')[1])
        data['month'] = data['month'].astype(float)
        data['day'] = data.Date.apply(lambda x: x.split('-')[2])
        data['day'] = data['day'].astype(float)

        # promo interval "Jan,Apr,Jul,Oct"
        data['promojan'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Jan" in x else 0)
        data['promofeb'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Feb" in x else 0)
        data['promomar'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Mar" in x else 0)
        data['promoapr'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Apr" in x else 0)
        data['promomay'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "May" in x else 0)
        data['promojun'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Jun" in x else 0)
        data['promojul'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Jul" in x else 0)
        data['promoaug'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Aug" in x else 0)
        data['promosep'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Sep" in x else 0)
        data['promooct'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Oct" in x else 0)
        data['promonov'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Nov" in x else 0)
        data['promodec'] = data.PromoInterval.apply(lambda x: 0 if isinstance(x, float) else 1 if "Dec" in x else 0)

    # # Features set.
    noisy_features = [myid,'Date']
    features = [c for c in features if c not in noisy_features]
    features_non_numeric = [c for c in features_non_numeric if c not in noisy_features]
    features.extend(['year','month','day'])

    # Fill NA
    class DataFrameImputer(TransformerMixin):
        # http://stackoverflow.com/questions/25239958/impute-categorical-missing-values-in-scikit-learn
        def __init__(self):
            """Impute missing values.
            Columns of dtype object are imputed with the most frequent value
            in column.
            Columns of other types are imputed with mean of column.
            """
        def fit(self, X, y=None):
            self.fill = pd.Series([X[c].value_counts().index[0] # mode
                if X[c].dtype == np.dtype('O') else X[c].mean() for c in X], # mean
                index=X.columns)
            return self
        def transform(self, X, y=None):
            return X.fillna(self.fill)
        
    train = DataFrameImputer().fit_transform(train)
    test = DataFrameImputer().fit_transform(test)
    # Pre-processing non-numberic values
    le = LabelEncoder()
    for col in features_non_numeric:
        le.fit(list(train[col])+list(test[col]))
        train[col] = le.transform(train[col])
        test[col] = le.transform(test[col])
    # LR和神经网络这种模型都对输入数据的幅度极度敏感，请先做归一化操作
#     scaler = StandardScaler()
#     for col in set(features) - set(features_non_numeric) - set([]): # TODO: add what not to scale
#         data_col = train[col] + test[col]
#         print(data_col.shape)
#         data_col.reshape(-1, 1)
#         print(data_col.shape)
#         scaler.fit(data_col)
#         train[col] = scaler.transform(train[col])
#         test[col] = scaler.transform(test[col])
    return (train,test,features,features_non_numeric)

### 训练与分析

In [36]:
def XGB_native(train,test,features,features_non_numeric):
    depth = 13
    eta = 0.01
    ntrees = 8000
    mcw = 3
    params = {"objective": "reg:linear",
              "booster": "gbtree",
              "eta": eta,
              "max_depth": depth,
              "min_child_weight": mcw,
              "subsample": 0.9,
              "colsample_bytree": 0.7,
              "silent": 1
              }
    print ("Running with params: " + str(params))
    print ("Running with ntrees: " + str(ntrees))
    print ("Running with features: " + str(features))

    # Train model with local split
    tsize = 0.05
    X_train, X_test = cross_validation.train_test_split(train, test_size=tsize)
    dtrain = xgb.DMatrix(X_train[features], np.log(X_train[goal] + 1))
    dvalid = xgb.DMatrix(X_test[features], np.log(X_test[goal] + 1))
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm = xgb.train(params, dtrain, ntrees, evals=watchlist, early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)
    train_probs = gbm.predict(xgb.DMatrix(X_test[features]))
    indices = train_probs < 0
    train_probs[indices] = 0
    error = rmspe(np.exp(train_probs) - 1, X_test[goal].values)
    print(error)

    # Predict and Export
    test_probs = gbm.predict(xgb.DMatrix(test[features]))
    indices = test_probs < 0
    test_probs[indices] = 0
    submission = pd.DataFrame({myid: test[myid], goal: np.exp(test_probs) - 1})
    if not os.path.exists('result/'):
        os.makedirs('result/')
        
    submission.to_csv("./result/dat-xgb_d%s_eta%s_ntree%s_mcw%s_tsize%s.csv" % (str(depth),str(eta),str(ntrees),str(mcw),str(tsize)) , index=None)
    
#     # Feature importance
#     if plot:
#       outfile = open('xgb.fmap', 'w')
#       i = 0
#       for feat in features:
#           outfile.write('{0}\t{1}\tq\n'.format(i, feat))
#           i = i + 1
#       outfile.close()
#       importance = gbm.get_fscore(fmap='xgb.fmap')
#       importance = sorted(importance.items(), key=operator.itemgetter(1))
#       df = pd.DataFrame(importance, columns=['feature', 'fscore'])
#       df['fscore'] = df['fscore'] / df['fscore'].sum()
#       # Plotitup
#       plt.figure()
#       df.plot()
#       df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(25, 15))
#       plt.title('XGBoost Feature Importance')
#       plt.xlabel('relative importance')
#       plt.gcf().savefig('Feature_Importance_xgb_d%s_eta%s_ntree%s_mcw%s_tsize%s.png' % (str(depth),str(eta),str(ntrees),str(mcw),str(tsize)))

In [ ]:
print ("=> 载入数据中...")
train,test,features,features_non_numeric = load_data()
print ("=> 处理数据与特征工程...")
train,test,features,features_non_numeric = process_data(train,test,features,features_non_numeric)
print ("=> 使用XGBoost建模...")
XGB_native(train,test,features,features_non_numeric)

=> 载入数据中...
=> 处理数据与特征工程...
=> 使用XGBoost建模...
Running with params: {'objective': 'reg:linear', 'booster': 'gbtree', 'eta': 0.01, 'max_depth': 13, 'min_child_weight': 3, 'subsample': 0.9, 'colsample_bytree': 0.7, 'silent': 1}
Running with ntrees: 8000
Running with features: ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'year', 'month', 'day']
[0]	eval-rmse:8.18658	train-rmse:8.18613	eval-rmspe:0.999864	train-rmspe:0.999864
Multiple eval metrics have been passed: 'train-rmspe' will be used for early stopping.

Will train until train-rmspe hasn't improved in 100 rounds.
[1]	eval-rmse:8.10487	train-rmse:8.10442	eval-rmspe:0.999838	train-rmspe:0.999838
[2]	eval-rmse:8.02399	train-rmse:8.02354	eval-rmspe:0.99981	train-rmspe:0.99981
[3]	eval-rmse:7.94392	train-rmse:7.94345	eval-rmspe:0.99978	train

[92]	eval-rmse:3.25991	train-rmse:3.25928	eval-rmspe:0.959265	train-rmspe:0.959241
[93]	eval-rmse:3.22755	train-rmse:3.22693	eval-rmspe:0.957933	train-rmspe:0.957908
[94]	eval-rmse:3.1955	train-rmse:3.19487	eval-rmspe:0.956577	train-rmspe:0.956548
[95]	eval-rmse:3.16379	train-rmse:3.16316	eval-rmspe:0.955188	train-rmspe:0.955156
[96]	eval-rmse:3.1324	train-rmse:3.13178	eval-rmspe:0.953769	train-rmspe:0.953734
[97]	eval-rmse:3.10135	train-rmse:3.10073	eval-rmspe:0.952314	train-rmspe:0.952275
[98]	eval-rmse:3.07057	train-rmse:3.06994	eval-rmspe:0.950842	train-rmspe:0.950798
[99]	eval-rmse:3.04012	train-rmse:3.03949	eval-rmspe:0.949333	train-rmspe:0.949285
[100]	eval-rmse:3.00998	train-rmse:3.00934	eval-rmspe:0.947793	train-rmspe:0.947742
[101]	eval-rmse:2.98015	train-rmse:2.97951	eval-rmspe:0.946223	train-rmspe:0.946167
[102]	eval-rmse:2.95059	train-rmse:2.94995	eval-rmspe:0.944627	train-rmspe:0.944566
[103]	eval-rmse:2.92133	train-rmse:2.92069	eval-rmspe:0.943001	train-rmspe:0.942934
[1

[190]	eval-rmse:1.23844	train-rmse:1.23776	eval-rmspe:0.704253	train-rmspe:0.698752
[191]	eval-rmse:1.22653	train-rmse:1.22584	eval-rmspe:0.700866	train-rmspe:0.695186
[192]	eval-rmse:1.21471	train-rmse:1.21402	eval-rmspe:0.6975	train-rmspe:0.691596
[193]	eval-rmse:1.20301	train-rmse:1.20232	eval-rmspe:0.694115	train-rmspe:0.688018
[194]	eval-rmse:1.19143	train-rmse:1.19073	eval-rmspe:0.690736	train-rmspe:0.684447
[195]	eval-rmse:1.17996	train-rmse:1.17926	eval-rmspe:0.687345	train-rmspe:0.680877
[196]	eval-rmse:1.16859	train-rmse:1.16788	eval-rmspe:0.684012	train-rmspe:0.677311
[197]	eval-rmse:1.15733	train-rmse:1.15662	eval-rmspe:0.680662	train-rmspe:0.673744
[198]	eval-rmse:1.14622	train-rmse:1.14551	eval-rmspe:0.677304	train-rmspe:0.670164
[199]	eval-rmse:1.13516	train-rmse:1.13446	eval-rmspe:0.673943	train-rmspe:0.666591
[200]	eval-rmse:1.12431	train-rmse:1.12359	eval-rmspe:0.670589	train-rmspe:0.663005
[201]	eval-rmse:1.11355	train-rmse:1.11284	eval-rmspe:0.667226	train-rmspe:0.6

[287]	eval-rmse:0.505921	train-rmse:0.504602	eval-rmspe:0.440877	train-rmspe:0.386072
[288]	eval-rmse:0.50169	train-rmse:0.500354	eval-rmspe:0.439259	train-rmspe:0.38362
[289]	eval-rmse:0.497547	train-rmse:0.496194	eval-rmspe:0.437624	train-rmspe:0.381202
[290]	eval-rmse:0.493395	train-rmse:0.492029	eval-rmspe:0.436058	train-rmspe:0.378802
[291]	eval-rmse:0.489214	train-rmse:0.487836	eval-rmspe:0.434479	train-rmspe:0.376387
[292]	eval-rmse:0.485081	train-rmse:0.483695	eval-rmspe:0.432935	train-rmspe:0.373992
[293]	eval-rmse:0.481095	train-rmse:0.479686	eval-rmspe:0.431443	train-rmspe:0.371546
[294]	eval-rmse:0.477066	train-rmse:0.47564	eval-rmspe:0.429959	train-rmspe:0.369158
[295]	eval-rmse:0.473033	train-rmse:0.471604	eval-rmspe:0.428493	train-rmspe:0.366813
[296]	eval-rmse:0.469073	train-rmse:0.467634	eval-rmspe:0.427038	train-rmspe:0.364483
[297]	eval-rmse:0.465189	train-rmse:0.463741	eval-rmspe:0.425664	train-rmspe:0.362192
[298]	eval-rmse:0.461429	train-rmse:0.459965	eval-rmspe:0

[383]	eval-rmse:0.257782	train-rmse:0.254829	eval-rmspe:0.37487	train-rmspe:0.232988
[384]	eval-rmse:0.25652	train-rmse:0.25354	eval-rmspe:0.374804	train-rmspe:0.232212
[385]	eval-rmse:0.255189	train-rmse:0.252181	eval-rmspe:0.374692	train-rmspe:0.231356
[386]	eval-rmse:0.253938	train-rmse:0.250898	eval-rmspe:0.374725	train-rmspe:0.230602
[387]	eval-rmse:0.252739	train-rmse:0.249674	eval-rmspe:0.374739	train-rmspe:0.229893
[388]	eval-rmse:0.251486	train-rmse:0.248396	eval-rmspe:0.374661	train-rmspe:0.229134
[389]	eval-rmse:0.250299	train-rmse:0.247191	eval-rmspe:0.374567	train-rmspe:0.228433
[390]	eval-rmse:0.249126	train-rmse:0.245994	eval-rmspe:0.374565	train-rmspe:0.227743
[391]	eval-rmse:0.247965	train-rmse:0.244804	eval-rmspe:0.374638	train-rmspe:0.226952
[392]	eval-rmse:0.246788	train-rmse:0.243603	eval-rmspe:0.374696	train-rmspe:0.226256
[393]	eval-rmse:0.245669	train-rmse:0.242456	eval-rmspe:0.374825	train-rmspe:0.225416
[394]	eval-rmse:0.244437	train-rmse:0.241209	eval-rmspe:0

...

[2005]	eval-rmse:0.096108	train-rmse:0.080056	eval-rmspe:0.362848	train-rmspe:0.095779
[2006]	eval-rmse:0.096098	train-rmse:0.080035	eval-rmspe:0.362841	train-rmspe:0.095755
[2007]	eval-rmse:0.096093	train-rmse:0.080025	eval-rmspe:0.36278	train-rmspe:0.095738
[2008]	eval-rmse:0.096081	train-rmse:0.080007	eval-rmspe:0.362777	train-rmspe:0.095719
[2009]	eval-rmse:0.096073	train-rmse:0.079993	eval-rmspe:0.362774	train-rmspe:0.095706
[2010]	eval-rmse:0.096069	train-rmse:0.079984	eval-rmspe:0.362771	train-rmspe:0.095696
[2011]	eval-rmse:0.096066	train-rmse:0.079978	eval-rmspe:0.362769	train-rmspe:0.09569
[2012]	eval-rmse:0.096058	train-rmse:0.079965	eval-rmspe:0.362383	train-rmspe:0.095678
[2013]	eval-rmse:0.09605	train-rmse:0.079956	eval-rmspe:0.362381	train-rmspe:0.095671
[2014]	eval-rmse:0.096039	train-rmse:0.079935	eval-rmspe:0.362336	train-rmspe:0.095644
[2015]	eval-rmse:0.096034	train-rmse:0.079927	eval-rmspe:0.362334	train-rmspe:0.095636
[2016]	eval-rmse:0.096023	train-rmse:0.079909	

[2100]	eval-rmse:0.095393	train-rmse:0.078812	eval-rmspe:0.361565	train-rmspe:0.094159
[2101]	eval-rmse:0.095383	train-rmse:0.078799	eval-rmspe:0.361539	train-rmspe:0.094146
[2102]	eval-rmse:0.095382	train-rmse:0.078793	eval-rmspe:0.361537	train-rmspe:0.09414
[2103]	eval-rmse:0.095371	train-rmse:0.07877	eval-rmspe:0.361524	train-rmspe:0.094112
[2104]	eval-rmse:0.095364	train-rmse:0.078754	eval-rmspe:0.361525	train-rmspe:0.094093
[2105]	eval-rmse:0.095354	train-rmse:0.078736	eval-rmspe:0.361516	train-rmspe:0.094056
[2106]	eval-rmse:0.095347	train-rmse:0.078717	eval-rmspe:0.361508	train-rmspe:0.09403
[2107]	eval-rmse:0.095345	train-rmse:0.078714	eval-rmspe:0.361507	train-rmspe:0.094027
[2108]	eval-rmse:0.095339	train-rmse:0.078706	eval-rmspe:0.361506	train-rmspe:0.094018
[2109]	eval-rmse:0.095336	train-rmse:0.0787	eval-rmspe:0.361505	train-rmspe:0.094009
[2110]	eval-rmse:0.095334	train-rmse:0.078691	eval-rmspe:0.361501	train-rmspe:0.093999
[2111]	eval-rmse:0.095319	train-rmse:0.078671	ev

[2195]	eval-rmse:0.09478	train-rmse:0.077674	eval-rmspe:0.360982	train-rmspe:0.092846
[2196]	eval-rmse:0.094773	train-rmse:0.077662	eval-rmspe:0.36098	train-rmspe:0.092833
[2197]	eval-rmse:0.094766	train-rmse:0.07765	eval-rmspe:0.360954	train-rmspe:0.092821
[2198]	eval-rmse:0.094757	train-rmse:0.077634	eval-rmspe:0.360936	train-rmspe:0.092805
[2199]	eval-rmse:0.094754	train-rmse:0.077627	eval-rmspe:0.360933	train-rmspe:0.092798
[2200]	eval-rmse:0.094752	train-rmse:0.077623	eval-rmspe:0.360933	train-rmspe:0.09279
[2201]	eval-rmse:0.094748	train-rmse:0.077606	eval-rmspe:0.360958	train-rmspe:0.092765
[2202]	eval-rmse:0.094747	train-rmse:0.077604	eval-rmspe:0.360957	train-rmspe:0.092763
[2203]	eval-rmse:0.094732	train-rmse:0.077582	eval-rmspe:0.360841	train-rmspe:0.092734
[2204]	eval-rmse:0.09473	train-rmse:0.077577	eval-rmspe:0.36084	train-rmspe:0.092635
[2205]	eval-rmse:0.094717	train-rmse:0.077557	eval-rmspe:0.36084	train-rmspe:0.092611
[2206]	eval-rmse:0.094713	train-rmse:0.077549	eval

[2290]	eval-rmse:0.094219	train-rmse:0.076542	eval-rmspe:0.35947	train-rmspe:0.091054
[2291]	eval-rmse:0.094214	train-rmse:0.076536	eval-rmspe:0.359464	train-rmspe:0.091048
[2292]	eval-rmse:0.094213	train-rmse:0.076532	eval-rmspe:0.35948	train-rmspe:0.091044
[2293]	eval-rmse:0.094208	train-rmse:0.076526	eval-rmspe:0.359479	train-rmspe:0.091037
[2294]	eval-rmse:0.094204	train-rmse:0.076503	eval-rmspe:0.359465	train-rmspe:0.09089
[2295]	eval-rmse:0.094202	train-rmse:0.0765	eval-rmspe:0.359462	train-rmspe:0.090887
[2296]	eval-rmse:0.094194	train-rmse:0.076485	eval-rmspe:0.359456	train-rmspe:0.090792
[2297]	eval-rmse:0.094192	train-rmse:0.076482	eval-rmspe:0.359459	train-rmspe:0.090789
[2298]	eval-rmse:0.094184	train-rmse:0.076466	eval-rmspe:0.359458	train-rmspe:0.090772
[2299]	eval-rmse:0.094177	train-rmse:0.076449	eval-rmspe:0.359466	train-rmspe:0.090754
[2300]	eval-rmse:0.094174	train-rmse:0.07644	eval-rmspe:0.359466	train-rmspe:0.090739
[2301]	eval-rmse:0.094168	train-rmse:0.07643	eval

[2385]	eval-rmse:0.093757	train-rmse:0.075552	eval-rmspe:0.359173	train-rmspe:0.089081
[2386]	eval-rmse:0.093746	train-rmse:0.075529	eval-rmspe:0.359167	train-rmspe:0.089052
[2387]	eval-rmse:0.093734	train-rmse:0.075504	eval-rmspe:0.359167	train-rmspe:0.089025
[2388]	eval-rmse:0.093723	train-rmse:0.075488	eval-rmspe:0.359166	train-rmspe:0.089009
[2389]	eval-rmse:0.09372	train-rmse:0.075482	eval-rmspe:0.359168	train-rmspe:0.089025
[2390]	eval-rmse:0.093712	train-rmse:0.075473	eval-rmspe:0.359165	train-rmspe:0.089016
[2391]	eval-rmse:0.093706	train-rmse:0.075464	eval-rmspe:0.359153	train-rmspe:0.089007
[2392]	eval-rmse:0.093705	train-rmse:0.075462	eval-rmspe:0.359156	train-rmspe:0.089006
[2393]	eval-rmse:0.093698	train-rmse:0.075445	eval-rmspe:0.359115	train-rmspe:0.088985
[2394]	eval-rmse:0.093689	train-rmse:0.075431	eval-rmspe:0.359099	train-rmspe:0.088973
[2395]	eval-rmse:0.093684	train-rmse:0.075421	eval-rmspe:0.359093	train-rmspe:0.088962
[2396]	eval-rmse:0.093677	train-rmse:0.07540